<a href="https://colab.research.google.com/github/babakansari/DeepLearning/blob/master/Countries_Autocomplete_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

Using TensorFlow backend.


In [4]:
import pandas as pd
import io

file_name = 'https://github.com/babakansari/DeepLearning/blob/master/Countries.xlsx?raw=true'

original_df = pd.read_excel(file_name)
original_df.head()


,Name,Ext
0,Afghanistan (AF),NaN
1,Aland Islands (AX),NaN
2,Albania (AL),NaN
3,Algeria (DZ),NaN
4,American Samoa (AS),NaN


In [5]:
df = pd.DataFrame(columns=['idx', 'Name', 'Source', 'Target', 'Len'])

idx = 0
for index, row in original_df.iterrows():
  name = row['Name']
  nameLen = row['Name'].find('(')

  for i, c in enumerate(name):
    if i == nameLen-2:
      break;
    df.loc[idx] = [idx, name[0:nameLen], (name[0:i]+c).rjust(40, '_'), name[i+1:nameLen].rjust(40, '_'), i+1]
    idx = idx + 1

df.head()

,idx,Name,Source,Target,Len
0,0,Afghanistan,_______________________________________A,_____________________________fghanistan,1
1,1,Afghanistan,______________________________________Af,______________________________ghanistan,2
2,2,Afghanistan,_____________________________________Afg,_______________________________hanistan,3
3,3,Afghanistan,____________________________________Afgh,________________________________anistan,4
4,4,Afghanistan,___________________________________Afgha,_________________________________nistan,5


In [0]:
#df.sort_values('Source').head(20)

In [0]:
encoded_df = pd.get_dummies(data=df, columns=['Source', 'Target'])

# src_df = pd.get_dummies(data=df, columns=['Source'])

# dest_df = pd.get_dummies(data=df, columns=['Target'])

# src_df

In [0]:
# Create list of Source and Target Columns
src_cols = []
trg_cols = []
for column in list(encoded_df.columns.values):
  if column.startswith("Source_"):
    src_cols.append(column)
  elif column.startswith("Target_"):
      trg_cols.append(column)

In [9]:
X = np.zeros((len(encoded_df), len(src_cols), len(trg_cols)), dtype=np.bool)

for index, row in encoded_df[:15].iterrows():
  for src_idx, src in enumerate(src_cols):
    if row[src] == 1:
      # print(str(index) + " = " + str(src_idx) )
      for trg_idx, trg in enumerate(trg_cols):
        if row[trg] == 1:
          #X[index, src_idx, trg_idx] = 1
          print(str(index) + " = " + str(src_idx) + ", " + str(trg_idx))

#X

0 = 1670, 1268
1 = 1556, 1208
2 = 1359, 1110
3 = 1145, 1005
4 = 951, 943
5 = 787, 761
6 = 657, 668
7 = 551, 483
8 = 459, 251
9 = 389, 228
10 = 1670, 1376
11 = 1557, 1315
12 = 1360, 1286
13 = 1146, 1196
14 = 952, 84


In [18]:
for index, row in encoded_df[:15].iterrows():
  name = row['Name']
  print(name)

Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Afghanistan 
Aland Islands 
Aland Islands 
Aland Islands 
Aland Islands 
Aland Islands 


In [13]:
src_df.shape

(2259, 1704)

In [13]:
name_enc.shape

(2259, 258)

## Model

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(2259, 1700)))

model.add(Dense(1700))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               936448    
_________________________________________________________________
dense_1 (Dense)              (None, 1700)              219300    
_________________________________________________________________
activation_4 (Activation)    (None, 1700)              0         
Total params: 1,155,748
Trainable params: 1,155,748
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

##Training

In [20]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(source_enc, name_enc, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

ValueError: ignored